In [1]:
#default_exp Clustering

In [2]:
#hide
from nbdev import *
from nbdev.showdoc import show_doc

In [3]:
#export
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from HierarchicalGeoClustering.Tree_clusters import cluster_node

# Clustering 

In this file all the clustering methods are implemented 


## Compute DBSCAN
A wapper function to obtain the DBSCAN cluters by levels

In [4]:
#export
def compute_dbscan(cluster,  **kwargs):
    
    """ 
    Sklearn DBSCAN wrapper.
    
    :param cluster: a (N,2) numpy array containing the obsevations

    :returns list with numpy arrays for all the clusters obtained
    """
    eps = kwargs.get( 'eps_DBSCAN',.04)
    debugg= kwargs.get( 'debugg',False)
    min_samples= kwargs.get( 'min_samples',50)
    ret_noise = kwargs.get('return_noise', False)
    # Standarize sample
    scaler = StandardScaler()
    cluster = scaler.fit_transform(cluster)
    if debugg:
        print('epsilon distance to DBSCAN: ', eps)
        print("min_samples to DBScan: ", min_samples )
        print("Number of points to fit the DBScan: ",cluster.shape[0])

    db = DBSCAN(eps=eps, min_samples=min_samples).fit(cluster)  # Check if can be run with n_jobs = -1
    
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    l_unique_labels = len(set(labels)) - (1 if -1 in labels else 0)
    unique_labels = set(labels) 
    cluster = scaler.inverse_transform(cluster)
    clusters = []
    if debugg:
        print('Number of clusters:' ,l_unique_labels)
    
    for l in unique_labels:
        if l != -1:
            class_member_mask = (labels == l)
            clusters.append(cluster[class_member_mask])
        elif l == -1 and debugg == True:
            class_member_mask = (labels == l)
            print("Muestras consideradas ruido: ",  sum(class_member_mask))
    
    if ret_noise == True:
        class_member_mask = (labels == -1)
        return clusters, cluster[class_member_mask]
    
    return clusters

In [5]:
#hide
### Pruebas 


## Recursive Clustering 
The function obtains the clustering iterative

In [6]:
#export
def recursive_clustering(
                this_level,  # Dictionary with Points
                to_process,  # levels to process
                cluster_tree,  # to store the clusters
                level = 0,  # current level
                algorithm ='dbscan',  # Algorithm to use
                **kwargs
               ):
    """
    Performs the recursive clustering.
    Calls compute_dbscan for each
    list of clusters keepen the structure and then calls itself
    until no more clusters satisfy the condition
        
    :param dict this_level: level is the current level 
    
    :param int to_process: the max level to process
    
    :param double eps: The epsilon parameter distance to pass to the needed algorithm 
    
    :param list cluster_tree : list of list to insert the levels 
    
    :param bool verbose : To print 
    
    :param double decay: In the use of dbscan the deacy parameter to reduce eps
    
    :param int min_points_cluster: The min point for each cluster to pass to algorithm
    
    :param str algorithm:  The string of the algorithm name to use
    """

    verbose= kwargs.get('verbose',False)
    min_points = kwargs.get( 'min_points_cluster', 50)
    decay = kwargs.get('decay', 0.7)
    eps = kwargs.get('eps' ,0.8)  # Epsilon distance to DBSCAN parameter
    tmp = None

    if level == 0:
        kwargs['eps'] = eps
    else:
        kwargs['eps'] = eps  * decay

    cluster_result_polygons = []
    if level > to_process:
        if verbose:
            print('Done clustering')
        return
    ######## Get the clusters for the current list of points 
    all_l = clustering(
                    this_level,
                    level=level,
                    algorithm=algorithm,
                    
                    **kwargs
                    )
    ##########

    cluster_tree.append(all_l)
    cluster_n = 0
    for i in all_l:
        cluster_n += len(i['points'])
    if verbose:
        print('At level ', level, ' the number of lists are ',
              len(all_l), ' with ', cluster_n, 'clusters')
    level += 1
    if len(all_l) > 0:
        return recursive_clustering(all_l, 
                               to_process=to_process,
                               cluster_tree=cluster_tree,
                               level= level,
                               algorithm=algorithm,
                               **kwargs
                               )
    else:
        if verbose:
            print('done clustering')
        return